## NK Model

In [1]:
#import necessary packages
import numpy as np
import random as rd
#set values of N and K here
N = 4
K = 2
X = 2**(K+1) #=total fitness contributions per gene (= columns in fitness matrix)

Creating 3 matrices with versions of NK model:
- NK - without neutrality
- NKp - with neutrality (probabilistic)
- NKq - with neutrality (quantised)

In [14]:
#create fitness matrix without neutrality: simply array of shape (N,X), filled with random decimals 
fmk = np.random.rand(N, X)

#NKp: reduce fraction of fitness contributions in fm to 0
# p = probability a value is set to 0
# Verel et al.: p ∈ {0.5, 0.8, 0.9}; Geard et al. p = ((N-1)/N) -  but explore options
p = ((N-1)/N) #is it? :)
fmp = np.where(np.random.rand(*fmk.shape) < p, 0, fmk)

#NKq: fitness contribution < 0.5 == 0, > 0.5 == 1
# q = number of quantiles (or levels). q > 1 - i.e. setting q as 2 will divide the decimals in two groups of integers: 0 and 1. 
# Verel et al. (2011) - q ∈ {2, 4, 10} - but explore options - Higher q -> lower neutrality 
q = 4
fmq = np.digitize(fmk, bins=np.linspace(0, 1, q+1), right=True) - 1

Choose neutrality version:

In [15]:
#replace fmk in following line with "fmp" or "fmq" when adding probabilistic or quantized neutrality to model
fm = np.copy(fmp) 


Creating corresponding epistasis matrix

In [16]:
#creates "identity matrix": array with genomes
im0 = np.arange(0, X, 1)
im1 = im0[np.newaxis, :]
im = np.repeat(im1, N, axis=0)
print("identity matrix")
print(im)

#Binary representation of im (just for visualisation)
imbin = np.vectorize(np.binary_repr)(im, 4) #increase to 8/16/32 with larger N 
print("binary identity matrix")
print(imbin)


identity matrix
[[0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]
 [0 1 2 3 4 5 6 7]]
binary identity matrix
[['0000' '0001' '0010' '0011' '0100' '0101' '0110' '0111']
 ['0000' '0001' '0010' '0011' '0100' '0101' '0110' '0111']
 ['0000' '0001' '0010' '0011' '0100' '0101' '0110' '0111']
 ['0000' '0001' '0010' '0011' '0100' '0101' '0110' '0111']]


In [17]:
#important: in this version each gene influenced by K others, but genes 
# can influence >2 other genes, so some are (way) more influential than others
# print a few times to see
val = list(range(1, N+1))
em1 = []
for row in range(1, N + 1):
    rd.shuffle(val)  # Shuffle the values
    em1.append(val[:2])  # Take the first two values to create a pair
em1 = np.array(em1)

In [18]:
#important: in this version each gene influenced by K others, and influences K others, so all equally influential
def generate_all_moved_permutation_tree(level, nums):
    if len(nums) == 0:
        raise RuntimeError('generate_permutation_tree must be called with a non-empty nums list')
    if len(nums) == 1:
        if level == nums[0]:
            return None
        else:
            return {nums[0]: {}}
    allowed_nums = list(nums)
    if level in allowed_nums:
        allowed_nums.remove(level)
    result = {}
    for n in allowed_nums:
        sublevel_nums = list(nums)
        if n in sublevel_nums:
            sublevel_nums.remove(n)
        subtree = generate_all_moved_permutation_tree(level + 1, sublevel_nums)
        if subtree is not None:
            result[n] = subtree
    if len(result) == 0:
        return None
    return result

def pick_an_all_moved_permutation(all_moved_permutation_tree, picked_numbers=None):
    if picked_numbers is None:
        picked_numbers = set()
    allowed_numbers = set(all_moved_permutation_tree.keys()) - picked_numbers
    if not allowed_numbers:
        return []
    
    n = rd.choice(list(allowed_numbers))
    picked_numbers.add(n)
    
    l = [n]
    sub_tree = all_moved_permutation_tree[n]
    
    if len(sub_tree) > 0:
        l.extend(pick_an_all_moved_permutation(sub_tree, picked_numbers))
    
    return l

def generate_unique_rows(t, num_rows):
    result = []
    for _ in range(num_rows):
        row = list(zip(pick_an_all_moved_permutation(t), pick_an_all_moved_permutation(t)))
        while any(x[0] == x[1] for x in row):
            row = list(zip(pick_an_all_moved_permutation(t), pick_an_all_moved_permutation(t)))
        result.extend(row)
    return np.array(result[:num_rows])

t = generate_all_moved_permutation_tree(1, range(1, 5))
em2 = generate_unique_rows(t, 4)

#for comparison: (can be removed)
print("Epistasis matrix with repetition")
print(em1)
print("Epistasis matrix without repetition")
print(em2)

Epistasis matrix with repetition
[[1 2]
 [3 1]
 [1 4]
 [3 4]]
Epistasis matrix without repetition
[[4 2]
 [1 3]
 [2 4]
 [3 1]]


Calculating coefficients ai0 to aij

In [19]:
def calc_a(K, fm): 
    a_coef = []
    for r in fm:
        a = [0.0] * X  # creates list with zeros as floats for each row & X cols
        a[0] = r[0] #because ai0=Fi0 # Calculate ai0 for i = 0
        for j in range(1, X): 
            sum = 0.0 
            for l in range(0, j): #only already calculated coeff
                if l == (l & j): #if l equal to bitwise AND of l and j (001&101->001 so TRUE, 001&100->000 so FALSE)
                    sum += a[l] 
            a[j] = r[j] - sum 
        a_coef.append(a) # append new a's into a_values array
    return a_coef

a_coef = calc_a(K, fm)

a_shape = np.reshape(a_coef, (N,X))

if np.array_equal(fm, fmk): 
    print("Coefficient matrix fmk")
elif np.array_equal(fm, fmp):
    print("Coefficient matrix fmp")
else: 
    print("Coefficient matrix fmq")
print(a_shape) 

Coefficient matrix fmp
[[ 0.          0.          0.          0.33746189  0.          0.
   0.         -0.33746189]
 [ 0.          0.          0.          0.49378642  0.3193987  -0.3193987
  -0.3193987  -0.17438773]
 [ 0.          0.          0.          0.          0.75378389 -0.75378389
  -0.75378389  1.23816875]
 [ 0.          0.          0.55014462 -0.55014462  0.          0.
  -0.55014462  0.55014462]]


Compute model for genome fitness

In [ ]:
#To-do
